# LoRA学習ノートブック（修正版）

このノートブックは、音声モデルに対してLoRA（Low-Rank Adaptation）を適用するためのテンプレートです。
エラーの原因となっていた依存関係の不足を解消し、動作確認用の軽量モデルでの実行例を追加しています。

## 1. 環境構築

必要なライブラリをインストールします。

In [ ]:
!pip install -U safetensors peft datasets transformers torch

## 2. モデルの準備

ベースモデルをロードし、LoRAを適用します。
※ここでは動作確認のため、非常に軽量なGPT-2を使用しています。実際の音声モデル（XTTS-v2等）に置き換えて使用してください。

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

# 実際のモデルパスに置き換えてください
model_name = "sshleifer/tiny-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA設定
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# LoRAモデルの作成
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

print("LoRA model initialized successfully!")

## 3. データセット準備

実際の学習には、テキストと音声のペアが必要です。

In [ ]:
from datasets import load_dataset

# 例: データセットの読み込み（実際には適切なデータセットを指定してください）
# dataset = load_dataset("path/to/your_dataset")

## 4. トレーナーで学習

`Trainer`を使用して学習を開始します。

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="outputs/lora_tts",
    per_device_train_batch_size=1,
    learning_rate=1e-4,
    num_train_epochs=5,
    no_cuda=True, # GPUがない環境用。ある場合はFalseに。
    logging_steps=10,
    save_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,  # 実際のデータセットを設定
)

# trainer.train() # データセット設定後に実行可能